In [12]:
from transformers import BertTokenizer
from transformers import BertModel
from torch import nn

# Defining the BERT model to generate the embeddings
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Defining the Triplet Loss with Euclidean Distance function
criteria = nn.TripletMarginLoss(margin=1.0, p=2)

def encode_sentence(sentence):
    tokens = tokenizer(sentence, add_special_tokens=True, return_tensors='pt', max_length=128, padding='max_length', truncation=True)
    return tokens['input_ids'], tokens['attention_mask']

def simple_forward(anchor, positive, negative):
    # Generating the tokens from our sentence
    input_ids1, attention_mask1 = encode_sentence(anchor)
    input_ids2, attention_mask2 = encode_sentence(positive)
    input_ids3, attention_mask3 = encode_sentence(negative)

    # Get the pooled output from the model for each instance
    anchoer_emb = model(input_ids1, attention_mask1)[1]
    positive_emb = model(input_ids2, attention_mask2)[1]
    negative_emb = model(input_ids3, attention_mask3)[1]

    # Compute the loss
    loss = criteria(anchoer_emb, positive_emb, negative_emb)
    return loss

first_sample = {
    "anchor": "I like trucks",
    "positive": "I love trucks",
    "negative": "Pigs can't fly"
}

second_sample = {
    "anchor": "I like trucks",
    "positive": "Pigs can't fly",
    "negative": "I love trucks"
}

simple_forward(**first_sample) # tensor(0., grad_fn=<MeanBackward0>)
simple_forward(**second_sample) # tensor(4.5003, grad_fn=<MeanBackward0>)

tensor(0., grad_fn=<MeanBackward0>)